In [ ]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=75c7e096bf5372632c0794f5dedd7a9883f5225b1677869404ad8f1393fd4653
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [2]:
!pip install numpy tqdm pillow torch torchvision torchaudio scikit-learn imbalanced-learn efficientnet_pytorch




Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------------------ --- 1.6/1.7 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   --------------------------------- ------ 2.1/2.5 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 10.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.1 MB 9.0 MB/s eta 0:00:02
   ------------- -------------------------- 3.7/11.1 MB 8.4 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.1 MB 7.4 MB/s eta 0:00:01
   ------------------ --------------------- 5.2/11.1 MB 6.2 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.1 MB 5.8 MB/s eta 0:00:01
   --------------------

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import ADASYN
from PIL import Image
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

In [2]:
# 1. Custom Dataset
class AlzheimerDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [3]:
# 2. Loading and preprocessing the data
def load_and_preprocess_data(data_dir):
    classes = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
    X = []
    y = []
    for i, class_name in enumerate(classes):
        for split in ['train', 'test']:
            class_dir = os.path.join(data_dir, split, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                img = Image.open(img_path).convert('RGB')
                X.append(img)
                y.append(i)
    return X, np.array(y)

In [4]:
import numpy as np
from imblearn.over_sampling import ADASYN
from PIL import Image

def apply_adasyn(X, y):
    # Convert PIL Images to numpy arrays
    X_arrays = [np.array(img) for img in X]
    
    # Get the shape of the images
    img_shape = X_arrays[0].shape
    
    # Reshape the arrays to 2D for ADASYN
    X_reshaped = np.array([x.flatten() for x in X_arrays])
    
    # Apply ADASYN
    adasyn = ADASYN(random_state=42)
    X_resampled, y_resampled = adasyn.fit_resample(X_reshaped, y)
    
    # Reshape back to original image shape
    X_balanced = [x.reshape(img_shape) for x in X_resampled]
    
    # Convert back to PIL Images
    X_balanced = [Image.fromarray(x.astype('uint8')) for x in X_balanced]
    
    return X_balanced, y_resampled

# The rest of the code remains the same

In [5]:
from imblearn.over_sampling import SMOTE

def apply_smote(X, y):
    # Convert PIL Images to numpy arrays
    X_arrays = [np.array(img) for img in X]
    
    # Get the shape of the images
    img_shape = X_arrays[0].shape
    
    # Reshape the arrays to 2D for SMOTE
    X_reshaped = np.array([x.flatten() for x in X_arrays])
    
    # Apply SMOTE
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_reshaped, y)
    
    # Reshape back to original image shape
    X_balanced = [x.reshape(img_shape) for x in X_resampled]
    
    # Convert back to PIL Images
    X_balanced = [Image.fromarray(x.astype('uint8')) for x in X_balanced]
    
    return X_balanced, y_resampled


In [6]:
# 4. Splitting the balanced data
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, stratify=y_train, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test


In [7]:
# 5. Creating the ensemble model
import torch
import torch.nn as nn
from torchvision import models
from efficientnet_pytorch import EfficientNet

class EnsembleModel(nn.Module):
    def __init__(self, num_classes=4):
        super(EnsembleModel, self).__init__()
        
        # Load pre-trained VGG16
        self.vgg16 = models.vgg16(pretrained=True)
        self.vgg16_features = nn.Sequential(*list(self.vgg16.features.children())[:-1])
        
        # Load pre-trained EfficientNet-B2
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b2')
        self.efficientnet._fc = nn.Identity()  # Remove the final fully connected layer
        
        # Adaptive pooling to ensure consistent spatial dimensions
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        
        # Combined feature processing
        self.dropout1 = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.bn1 = nn.BatchNorm1d(7 * 7 * 1920)
        self.fc1 = nn.Linear(7 * 7 * 1920, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        # VGG16 feature extraction
        vgg_features = self.vgg16_features(x)
        vgg_features = self.adaptive_pool(vgg_features)
        
        # EfficientNet-B2 feature extraction
        efficientnet_features = self.efficientnet.extract_features(x)
        efficientnet_features = self.adaptive_pool(efficientnet_features)
        
        # Concatenate features
        combined_features = torch.cat((vgg_features, efficientnet_features), dim=1)
        
        # Process combined features
        x = self.dropout1(combined_features)
        x = self.flatten(x)
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.bn3(x)
        x = self.fc2(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        return x

# Replace the create_model function with this:
def create_model(num_classes=4):
    return EnsembleModel(num_classes)

In [8]:
# 6. Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=50, device='cuda'):
    best_val_loss = float('inf')
    patience = 5
    counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_correct / train_total
        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_correct / val_total
        
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping after {epoch+1} epochs')
                break
    
    return model

In [9]:
# 7. Testing function
def test_model(model, test_loader, criterion, device='cuda'):
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()
    
    test_loss = test_loss / len(test_loader.dataset)
    test_acc = test_correct / test_total
    
    print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

In [10]:
data_dir = r"C:\Users\dell\OneDrive\Desktop\alzheimers-disease-detection-main\alzheimers-disease-detection-main\Combined Dataset"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# Load and preprocess data
X, y = load_and_preprocess_data(data_dir)

In [12]:
len(X)

2998

In [13]:
len(y)

2998

In [14]:
# Apply ADASYN
X_balanced, y_balanced = apply_smote(X, y)

In [15]:
len(X_balanced)

4324

In [16]:
len(y_balanced)

4324

In [17]:
# Split data
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_balanced, y_balanced)

In [18]:
# Define transforms
transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [19]:
batch_size = 32

# Create datasets
train_dataset = AlzheimerDataset(X_train, y_train, transform=transform)
val_dataset = AlzheimerDataset(X_val, y_val, transform=transform)
test_dataset = AlzheimerDataset(X_test, y_test, transform=transform)
    
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)    

In [20]:
# Create model
model = create_model().to(device)

C:\Users\dell\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dell\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded pretrained weights for efficientnet-b2


In [21]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)    

In [ ]:
# Train model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler)

Epoch 1/50:   0%|          | 0/95 [00:00<?, ?it/s]

In [27]:
# Load best model and test
best_model = create_model().to(device)
best_model.load_state_dict(torch.load('best_model.pth'))
test_model(best_model, test_loader, criterion)

Loaded pretrained weights for efficientnet-b2


/tmp/ipykernel_30/4122861454.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load('best_model.pth'))


Test Loss: 0.0753, Test Acc: 0.9742
